In [7]:
# --- Block 1: Install Dependencies ---
print("--- Installing 'lwcc' (LightWeight Crowd Counting)... ---")
!pip install lwcc -q
print("--- Installation complete. ---")

--- Installing 'lwcc' (LightWeight Crowd Counting)... ---
--- Installation complete. ---
--- Installation complete. ---


In [8]:

# --- Block 2: Import Libraries ---
print("\n--- Importing necessary libraries... ---")
import os
import glob
import csv
# google.colab may not be available when running locally or in CI
try:
    from google.colab import drive
    HAVE_COLAB = True
except Exception:
    HAVE_COLAB = False
# lwcc is optional (fallback); guard import
try:
    from lwcc import LWCC
except Exception:
    LWCC = None
from PIL import Image

print("--- Imports successful (local/CI safe). ---")


--- Importing necessary libraries... ---


/Users/mayanksoni/Desktop/crowd/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


--- Imports successful (local/CI safe). ---


In [9]:
# --- Block 3: Mount Google Drive (safe) ---
print("\n--- Checking for Google Drive (safe) ... ---")
if HAVE_COLAB:
    try:
        drive.mount('/content/drive', force_remount=True)
        print("--- Drive mounted successfully. ---")
    except Exception as e:
        print(f"Could not mount Drive: {e}")
else:
    print("Google Colab Drive not available; running locally/CI.")

# --- Block 4: Configuration ---
BASE_DRIVE_FOLDER = os.environ.get('DRIVE_PATH', '/content/drive/MyDrive/sample_crowd_management/samplecrowd')
OUTPUT_CSV_PATH = os.environ.get('OUTPUT_CSV_PATH', './outputs/predictions_summary.csv')

# choose drive folder if exists else fallback to repo sample images
if os.path.isdir(BASE_DRIVE_FOLDER):
    chosen_base = BASE_DRIVE_FOLDER
elif os.path.isdir('./data/sample_images'):
    chosen_base = './data/sample_images'
else:
    chosen_base = BASE_DRIVE_FOLDER  # will raise later if missing

print(f"Base image folder (chosen): {chosen_base}")
print(f"Output CSV will be: {OUTPUT_CSV_PATH}")

# --- Block 5: Find All Images Recursively ---
print("\n--- Searching for images in nested folders... ---")
image_extensions = ('*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tif', '*.tiff')
image_list = []
for ext in image_extensions:
    image_list.extend(glob.glob(os.path.join(chosen_base, '**', ext), recursive=True))
if not image_list:
    raise ValueError(f"No images found in '{chosen_base}'. Please add images under './data/sample_images' or set DRIVE_PATH environment variable.")

print(f"--- Found {len(image_list)} images. ---")

# --- Block 6: Predict Crowd Count INDIVIDUALLY ---
print("\n--- Predicting crowd count image-by-image... ---")
MODEL_NAME = "DM-Count"   # Can also be "CSRNet", "Bayesian", "SFANet"
MODEL_WEIGHTS = "SHA"     # Use "SHA" for dense, "SHB" for sparse
RESIZE_IMG = True

crowd_counts = {}
for img_path in image_list:
    try:
        if LWCC is None:
            raise RuntimeError('LWCC not available. Please install lwcc or provide a torch model.')
        count = LWCC.get_count(
            [img_path],
            model_name=MODEL_NAME,
            model_weights=MODEL_WEIGHTS,
            resize_img=RESIZE_IMG
        )
        crowd_counts[img_path] = int(round(count))
    except Exception as e:
        print(f"[!] Error processing {os.path.basename(img_path)}: {e}")
        crowd_counts[img_path] = "ERROR"

print("--- Prediction complete. ---")

# --- Block 7: Save to CSV ---
print(f"\n--- Saving results to {OUTPUT_CSV_PATH}... ---")
os.makedirs(os.path.dirname(OUTPUT_CSV_PATH), exist_ok=True)
with open(OUTPUT_CSV_PATH, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['ImagePath', 'RelativePath', 'CrowdCount'])
    for img_path, count in crowd_counts.items():
        rel_path = os.path.relpath(img_path, chosen_base)
        writer.writerow([img_path, rel_path, count])

print("--- CSV file saved. ---")

# --- Block 8: Display Sample Results ---
print("\n--- RESULTS SAMPLE ---\n")
for i, (img, ct) in enumerate(list(crowd_counts.items())):
    print(f"{os.path.basename(img)}: {ct} people (estimated)")
print("----\nDone.")


--- Checking for Google Drive (safe) ... ---
Google Colab Drive not available; running locally/CI.
Base image folder (chosen): /content/drive/MyDrive/sample_crowd_management/samplecrowd
Output CSV will be: ./outputs/predictions_summary.csv

--- Searching for images in nested folders... ---


ValueError: No images found in '/content/drive/MyDrive/sample_crowd_management/samplecrowd'. Please add images under './data/sample_images' or set DRIVE_PATH environment variable.

In [12]:
# --- Drive-aware inference runner (clean, robust) ---
print('\n--- Running robust inference runner ---')
import os, sys, random, json, csv
from pathlib import Path

# prefer helper utilities if available
try:
    from inference_utils import find_image_paths, load_model, safe_inference_loop, visualize_and_save, set_seeds
    helper_ok = True
except Exception:
    helper_ok = False

# torch availability
try:
    import torch
    has_torch = True
except Exception:
    has_torch = False

# reproducibility
SEED = 42
random.seed(SEED)
try:
    import numpy as np
    np.random.seed(SEED)
except Exception:
    np = None
if helper_ok:
    set_seeds(SEED)

# detect images (Drive-aware + repo fallbacks)
try:
    image_paths, src = find_image_paths('./data/sample_images') if helper_ok else (None, None)
except Exception as e:
    image_paths, src = None, None

# If helper not present or didn't find images, try fallback detection (repo 'samplecrowd')
if not image_paths:
    cand = Path(__file__).resolve().parents[1] / 'samplecrowd'
    if cand.exists():
        import glob as _glob
        image_paths = sorted([str(p) for p in cand.glob('*') if p.suffix.lower() in ('.jpg','.jpeg','.png','.bmp','.tif','.tiff')])
        src = str(cand)

if not image_paths:
    raise RuntimeError('No images found in Drive, ./data/sample_images, or repo samplecrowd; please add images or set DRIVE_PATH')

print(f'Found {len(image_paths)} images (source: {src})')

# outputs
OUT_DIR = Path('./outputs')
OUT_DIR.mkdir(parents=True, exist_ok=True)
SUMMARY_CSV = OUT_DIR / 'predictions_summary.csv'

# model loading: try torch model then LWCC fallback
MODEL_PATH = Path(os.getenv('MODEL_PATH', './model_weights.pt'))
model = None
device = 'cpu'
if has_torch:
    try:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        if helper_ok:
            model = load_model(str(MODEL_PATH), device=device)
        else:
            if MODEL_PATH.exists():
                try:
                    model = torch.jit.load(str(MODEL_PATH), map_location=device)
                except Exception:
                    model = torch.load(str(MODEL_PATH), map_location=device)
                try:
                    model.eval()
                    model.to(device)
                except Exception:
                    pass
    except Exception as e:
        print('Model load error:', e)
else:
    print('torch not installed; will try LWCC fallback')

USE_LWCC = False
if model is None:
    try:
        from lwcc import LWCC
        USE_LWCC = True
        print('LWCC available — using LWCC fallback')
    except Exception:
        if not has_torch:
            print('No torch and no LWCC — install one to run inference: pip install torch torchvision OR pip install lwcc')

# run inference
results = []
try:
    from tqdm import tqdm
except Exception:
    def tqdm(x, **kw): return x

if model is not None:
    print('Running model inference on images...')
    results = safe_inference_loop(model, image_paths, device=device, target_size=(512,512), outputs_dir=str(OUT_DIR)) if helper_ok else None
    if results is None:
        # fallback simple loop if helper not present
        for p in tqdm(image_paths, desc='Infer'):
            try:
                # minimal preprocessing: use PIL -> tensor
                from PIL import Image as _Image
                img = _Image.open(p).convert('RGB')
                import numpy as _np
                arr = _np.array(img.resize((512,512))).astype('float32')/255.0
                arr = _np.transpose(arr, (2,0,1))
                t = torch.from_numpy(arr).unsqueeze(0).to(device)
                with torch.no_grad(): out = model(t)
                try:
                    out_arr = out.detach().cpu().numpy()
                    cnt = float(out_arr.sum()) if out_arr.ndim>=2 else float(out_arr.mean())
                except Exception:
                    try: cnt = float(out)
                    except Exception: cnt = 0.0
                visualize_and_save(p, cnt, outputs_dir=str(OUT_DIR)) if helper_ok else None
                results.append({'image': p, 'count': int(round(cnt))})
            except Exception as e:
                print(f'Error on {p}: {e}')
                results.append({'image': p, 'count': None, 'error': str(e)})

elif USE_LWCC:
    print('Running LWCC fallback inference...')
    for p in tqdm(image_paths, desc='LWCC'):
        try:
            c = LWCC.get_count([p], model_name='DM-Count', model_weights='SHA', resize_img=True)
            c_int = int(round(float(c)))
            try:
                visualize_and_save(p, c_int, outputs_dir=str(OUT_DIR)) if helper_ok else None
            except Exception:
                pass
            results.append({'image': p, 'count': c_int})
        except Exception as e:
            print(f'LWCC error on {p}: {e}')
            results.append({'image': p, 'count': None, 'error': str(e)})
else:
    raise RuntimeError('No inference backend available (torch model missing and LWCC not present)')

# write summary CSV
with open(SUMMARY_CSV, 'w', newline='', encoding='utf-8') as fh:
    writer = csv.writer(fh)
    writer.writerow(['image', 'prediction_count'])
    for r in results:
        writer.writerow([r.get('image'), r.get('count')])

print(f'\nSaved {len(results)} results to: {OUT_DIR.resolve()}')
print(f'Summary CSV: {SUMMARY_CSV.resolve()}')

# smoke test: show first two results
print('\nSmoke test (first 2 results):', results[:2])
print('\n--- Runner finished ---')


--- Running robust inference runner ---
Found 13 images (source: /Users/mayanksoni/Desktop/crowd/Crowd_Management_System/samplecrowd)
Model weights not found at 'model_weights.pt'. Skipping heavy inference.
LWCC available — using LWCC fallback
Running LWCC fallback inference...


LWCC: 100%|██████████| 13/13 [00:00<00:00, 7440.77it/s]

LWCC error on /Users/mayanksoni/Desktop/crowd/Crowd_Management_System/samplecrowd/-Blvd-Antonio-l-Rodriguez-188-Conference-Room-Member-Engagement-1-1440x810_jpg.rf.83d47fa9453b297294d9dddfea0640c7.jpg: [Errno 30] Read-only file system: '/.lwcc'
LWCC error on /Users/mayanksoni/Desktop/crowd/Crowd_Management_System/samplecrowd/1.jpg: [Errno 30] Read-only file system: '/.lwcc'
LWCC error on /Users/mayanksoni/Desktop/crowd/Crowd_Management_System/samplecrowd/10.jpg: [Errno 30] Read-only file system: '/.lwcc'
LWCC error on /Users/mayanksoni/Desktop/crowd/Crowd_Management_System/samplecrowd/2.jpg: [Errno 30] Read-only file system: '/.lwcc'
LWCC error on /Users/mayanksoni/Desktop/crowd/Crowd_Management_System/samplecrowd/3.jpg: [Errno 30] Read-only file system: '/.lwcc'
LWCC error on /Users/mayanksoni/Desktop/crowd/Crowd_Management_System/samplecrowd/4.jpg: [Errno 30] Read-only file system: '/.lwcc'
LWCC error on /Users/mayanksoni/Desktop/crowd/Crowd_Management_System/samplecrowd/5.jpg: [Errn